In [1]:
import pandas as pd
from click import progressbar
from modin.core.execution.modin_aqp import progress_bars
from sklearn.utils import resample
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split, RandomizedSearchCV, cross_val_predict
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
import numpy as np
from sklearn.impute import SimpleImputer
import joblib
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
import optuna
from optuna.samplers import TPESampler
from sklearn.linear_model import LogisticRegression


In [2]:
def get_oof_predictions(model, X, y, cat_features=None, n_folds=5):
    """
    Genera predicciones out-of-fold para stacking
    """
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    oof_preds = np.zeros(len(X))

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        print(f"Fold {fold + 1}/{n_folds}")

        X_train_fold = X.iloc[train_idx]
        y_train_fold = y.iloc[train_idx]
        X_val_fold = X.iloc[val_idx]

        # Clonar el modelo para cada fold
        if isinstance(model, CatBoostClassifier):
            fold_model = CatBoostClassifier(**model.get_params())
            fold_model.fit(
                X_train_fold, y_train_fold,
                eval_set=(X_val_fold, y.iloc[val_idx]),
                cat_features=cat_features,
                verbose=0
            )
            oof_preds[val_idx] = fold_model.predict_proba(X_val_fold)[:, 1]

        elif isinstance(model, XGBClassifier):
            fold_model = XGBClassifier(**model.get_params())
            fold_model.fit(
                X_train_fold, y_train_fold,
                eval_set=[(X_val_fold, y.iloc[val_idx])],
                verbose=0
            )
            oof_preds[val_idx] = fold_model.predict_proba(X_val_fold)[:, 1]


    return oof_preds

In [3]:
#load dataset
df = pd.read_csv("../data/processed/home_credit_train_ready.csv")

In [4]:

#rebalance dataset
class_0 = df[df['TARGET'] == 0]
class_1 = df[df['TARGET'] == 1]

# balance mayority class with minority
sub_class = resample(class_0, replace=False, n_samples=(len(class_1) * 1), random_state=42)

# combine
df = pd.concat([sub_class, class_1])

In [5]:
y = df['TARGET']
X = df.drop(columns=['TARGET', 'SK_ID_CURR'])  #adde sk_id_curr

In [6]:
#manages divided by 0 values
X.replace([np.inf, -np.inf], np.nan, inplace=True)

In [7]:
#divide categorical and numerical to impute the values
cat_cols = X.select_dtypes(include=["object"]).columns
num_cols = X.select_dtypes(exclude=["object"]).columns

num_imputer = SimpleImputer(strategy="constant", fill_value= 0)  #fill nan's as 0
cat_imputer = SimpleImputer(strategy="constant", fill_value="missing")

X[num_cols] = num_imputer.fit_transform(X[num_cols])
X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])

In [8]:
#change object to category to use in the threelike model
for col in X.select_dtypes("object"):
    X[col] = X[col].astype("category")

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y ,random_state=42)

In [10]:
# Identificar columnas categóricas
cat_features = X_train.select_dtypes(include=['category', 'object']).columns.tolist()

In [11]:
#catboost
# Calcula el ratio de desbalance
neg_samples = (y_train == 0).sum()
pos_samples = (y_train == 1).sum()
scale_pos_weight = neg_samples / pos_samples

catboost_params = {
    'loss_function': 'Logloss',
    'eval_metric': 'PRAUC',

    'scale_pos_weight': scale_pos_weight * 0.9,

    # Boosting
    'iterations': 4000,
    'learning_rate': 0.025,
    'depth': 6,

    # Regularización (compensa falta de rsm)
    'l2_leaf_reg': 8.0,
    'random_strength': 2,

    # Bootstrap
    'bootstrap_type': 'Bayesian',
    'bagging_temperature': 0.5,

    # Growing
    'grow_policy': 'SymmetricTree',
    'min_data_in_leaf': 30,

    # Categóricas
    'cat_features': cat_features,
    'one_hot_max_size': 10,

    # GPU
    'task_type': 'GPU',
    'devices': '0',

    # Control
    'random_seed': 42,

    'verbose': 100,}

In [12]:
#xgb
xgb_params = {
    # Objective y métrica
    'objective': 'binary:logistic',
    'eval_metric': 'aucpr',  # AUCPR es mejor para desbalanceo


    # Parámetros de boosting - MÁS AGRESIVOS
    'n_estimators': 3000,  # Reducir de 5000 (early stopping lo maneja)
    'learning_rate': 0.01,  # Más bajo = más refinado
    'max_depth': 6,  # Aumentar de 5 (más capacidad)

    # Regularización - BALANCEADA
    'min_child_weight': 5,  # Reducir de 7 (menos restrictivo)
    'gamma': 0.2,  # Reducir de 0.3
    'subsample': 0.8,  # Aumentar de 0.7
    'colsample_bytree': 0.8,  # Aumentar de 0.7
    'colsample_bylevel': 0.8,  # Aumentar de 0.7
    'colsample_bynode': 0.8,  # NUEVO: muestreo por nodo

    # Regularización L1/L2 - MÁS SUAVE
    'reg_alpha': 0.5,  # Reducir de 1.0
    'reg_lambda': 2.0,  # Reducir de 3.0

    # Parámetros adicionales IMPORTANTES
    'max_delta_step': 1,  # Ayuda con desbalanceo extremo
    'tree_method': 'hist',
    'grow_policy': 'depthwise',  # vs lossguide

    # Sampling method para desbalanceo
    'sampling_method': 'gradient_based',  # Alternativa: 'uniform'

    # Control
    'random_state': 42,
    'n_jobs': -1,
    'device': 'cuda',
      # Reducir de 300 (más rápido)
    'verbosity': 1,
    'enable_categorical': True,
}

In [13]:
catboost = CatBoostClassifier(**catboost_params)
xgboost = XGBClassifier(**xgb_params)


In [14]:
print("\n--- CatBoost ---")
cat_oof = get_oof_predictions(catboost, X_train, y_train, cat_features=cat_features, n_folds=5)

print("\n--- XGBoost ---")
xgb_oof = get_oof_predictions(xgboost, X_train, y_train, cat_features=cat_features, n_folds=5)


--- CatBoost ---
Fold 1/5


Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


Fold 2/5


Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


Fold 3/5


...

Fold 5/5


Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time



--- XGBoost ---
Fold 1/5


Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.



Fold 2/5
Fold 3/5
Fold 4/5
Fold 5/5


In [15]:
meta_features_train = np.column_stack([cat_oof, xgb_oof])

In [16]:
meta_model = LogisticRegression(max_iter=5000, solver='saga', random_state=42, n_jobs=-1)
meta_model.fit(meta_features_train, y_train)

LogisticRegression(max_iter=5000, n_jobs=-1, random_state=42, solver='saga')

In [17]:
meta_train_preds = meta_model.predict_proba(meta_features_train)[:, 1]
print(f"AUC Meta-modelo (train): {roc_auc_score(y_train, meta_train_preds):.4f}")


AUC Meta-modelo (train): 0.7843


In [18]:
catboost_final = CatBoostClassifier(**catboost_params)
catboost_final.fit(X_train, y_train, cat_features=cat_features, verbose=100)

xgboost_final = XGBClassifier(**xgb_params)
xgboost_final.fit(X_train, y_train, verbose=100)


Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.6959929	total: 24.7ms	remaining: 1m 38s
100:	learn: 0.7381365	total: 2.03s	remaining: 1m 18s
200:	learn: 0.7558326	total: 3.99s	remaining: 1m 15s
300:	learn: 0.7666326	total: 6.12s	remaining: 1m 15s
400:	learn: 0.7752292	total: 8.39s	remaining: 1m 15s
500:	learn: 0.7834180	total: 10.7s	remaining: 1m 14s
600:	learn: 0.7912101	total: 13.3s	remaining: 1m 15s
700:	learn: 0.7982624	total: 15.7s	remaining: 1m 13s
800:	learn: 0.8047149	total: 17.6s	remaining: 1m 10s
900:	learn: 0.8110950	total: 19.5s	remaining: 1m 7s
1000:	learn: 0.8172388	total: 21.5s	remaining: 1m 4s
1100:	learn: 0.8231289	total: 23.3s	remaining: 1m 1s
1200:	learn: 0.8287229	total: 25.1s	remaining: 58.5s
1300:	learn: 0.8340760	total: 26.9s	remaining: 55.9s
1400:	learn: 0.8392877	total: 28.8s	remaining: 53.3s
1500:	learn: 0.8444261	total: 30.6s	remaining: 50.9s
1600:	learn: 0.8493444	total: 32.4s	remaining: 48.5s
1700:	learn: 0.8542314	total: 34.2s	remaining: 46.2s
1800:	learn: 0.8587148	total: 36s	remaining: 44s

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.8, colsample_bynode=0.8, colsample_bytree=0.8,
              device='cuda', early_stopping_rounds=None,
              enable_categorical=True, eval_metric='aucpr', feature_types=None,
              feature_weights=None, gamma=0.2, grow_policy='depthwise',
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=1, max_depth=6,
              max_leaves=None, min_child_weight=5, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=3000,
              n_jobs=-1, num_parallel_tree=None, ...)

In [19]:
test_cat_preds = catboost_final.predict_proba(X_test)[:, 1]
test_xgb_preds = xgboost_final.predict_proba(X_test)[:, 1]

meta_features_test = np.column_stack([test_cat_preds, test_xgb_preds])

In [20]:
final_predictions = meta_model.predict_proba(meta_features_test)[:, 1]

In [22]:
print(f"\nAUC CatBoost (test): {roc_auc_score(y_test, test_cat_preds):.4f}")
print(f"AUC XGBoost (test): {roc_auc_score(y_test, test_xgb_preds):.4f}")
print(f"AUC STACKING (test): {roc_auc_score(y_test, final_predictions):.4f}")


AUC CatBoost (test): 0.7852
AUC XGBoost (test): 0.7858
AUC STACKING (test): 0.7871


In [25]:
print("\n" + "="*60)
print("EVALUACIÓN CON THRESHOLD OPTIMIZADO")
print("="*60)

# Costos de negocio
cost_FN = 10000  # Costo de no detectar un default (pérdida del préstamo)
cost_FP = 1000   # Costo de rechazar un buen cliente (oportunidad perdida)

# Buscar threshold óptimo
thresholds = np.linspace(0, 1, 101)
losses = []

for t in thresholds:
    y_pred = (final_predictions >= t).astype(int)

    FP = np.sum((y_test == 0) & (y_pred == 1))
    FN = np.sum((y_test == 1) & (y_pred == 0))

    loss = FP * cost_FP + FN * cost_FN
    losses.append(loss)

optimal_threshold = thresholds[np.argmin(losses)]
min_loss = min(losses)

print(f"\n💰 Análisis de Costos:")
print(f"Threshold óptimo: {optimal_threshold:.4f}")
print(f"Pérdida mínima: ${min_loss:,.0f}")

# Predicciones con threshold optimizado
y_test_pred = (final_predictions >= optimal_threshold).astype(int)

# Métricas de clasificación
print(f"\n📈 Métricas de Clasificación (threshold={optimal_threshold:.4f}):")

# Confusion matrix manual
TP = np.sum((y_test == 1) & (y_test_pred == 1))
FP = np.sum((y_test == 0) & (y_test_pred == 1))
TN = np.sum((y_test == 0) & (y_test_pred == 0))
FN = np.sum((y_test == 1) & (y_test_pred == 0))

FPR = FP / (FP + TN) if (FP + TN) > 0 else 0
TPR = TP / (TP + FN) if (TP + FN) > 0 else 0
Precision = TP / (TP + FP) if (TP + FP) > 0 else 0
Recall = TPR

print(f"FPR (False Positive Rate): {FPR:.4f}")
print(f"TPR (Recall): {TPR:.4f}")
print(f"Precision (PPV): {Precision:.4f}")
print(f"AUC: {roc_auc_score(y_test, final_predictions):.5f}")

# Classification report
print(f"\n📋 Classification Report:")
print(classification_report(y_test, y_test_pred, target_names=['No Default', 'Default']))

# Confusion matrix
cm = confusion_matrix(y_test, y_test_pred)
print(f"\n📊 Matriz de Confusión:")
print(f"                 Predicho")
print(f"               No Def  Default")
print(f"Real No Def    {cm[0, 0]:6d}   {cm[0, 1]:6d}")
print(f"Real Default   {cm[1, 0]:6d}   {cm[1, 1]:6d}")

# Análisis de costos detallado
total_cost_optimal = FP * cost_FP + FN * cost_FN
total_cost_default = np.sum(y_test == 1) * cost_FN  # Si rechazamos a todos
savings = total_cost_default - total_cost_optimal

print(f"\n💵 Análisis Financiero:")
print(f"Costo total (threshold óptimo): ${total_cost_optimal:,.0f}")
print(f"Costo si rechazamos a todos: ${total_cost_default:,.0f}")
print(f"Ahorro con el modelo: ${savings:,.0f} ({(savings/total_cost_default)*100:.1f}%)")
print(f"\nFalsos Positivos: {FP} (costo: ${FP * cost_FP:,.0f})")
print(f"Falsos Negativos: {FN} (costo: ${FN * cost_FN:,.0f})")


EVALUACIÓN CON THRESHOLD OPTIMIZADO

💰 Análisis de Costos:
Threshold óptimo: 0.1000
Pérdida mínima: $8,034,000

📈 Métricas de Clasificación (threshold=0.1000):
FPR (False Positive Rate): 0.7996
TPR (Recall): 0.9760
Precision (PPV): 0.3790
AUC: 0.78713

📋 Classification Report:
              precision    recall  f1-score   support

  No Default       0.94      0.20      0.33      8734
     Default       0.38      0.98      0.55      4367

    accuracy                           0.46     13101
   macro avg       0.66      0.59      0.44     13101
weighted avg       0.76      0.46      0.40     13101


📊 Matriz de Confusión:
                 Predicho
               No Def  Default
Real No Def      1750     6984
Real Default      105     4262

💵 Análisis Financiero:
Costo total (threshold óptimo): $8,034,000
Costo si rechazamos a todos: $43,670,000
Ahorro con el modelo: $35,636,000 (81.6%)

Falsos Positivos: 6984 (costo: $6,984,000)
Falsos Negativos: 105 (costo: $1,050,000)


In [26]:
df = pd.read_csv("../data/processed/home_credit_train_ready.csv")

y = df['TARGET']
X = df.drop(columns=['TARGET', 'SK_ID_CURR'])  #adde sk_id_curr

X.replace([np.inf, -np.inf], np.nan, inplace=True)

#divide categorical and numerical to impute the values
cat_cols = X.select_dtypes(include=["object"]).columns
num_cols = X.select_dtypes(exclude=["object"]).columns

num_imputer = SimpleImputer(strategy="constant", fill_value= 0)  #fill nan's as 0
cat_imputer = SimpleImputer(strategy="constant", fill_value="missing")

X[num_cols] = num_imputer.fit_transform(X[num_cols])
X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])

#change object to category to use in the threelike model
for col in X.select_dtypes("object"):
    X[col] = X[col].astype("category")

In [27]:
test_cat_preds = catboost_final.predict_proba(X)[:, 1]
test_xgb_preds = xgboost_final.predict_proba(X)[:, 1]

meta_features_test = np.column_stack([test_cat_preds, test_xgb_preds])


In [28]:
final_predictions = meta_model.predict_proba(meta_features_test)[:, 1]


In [29]:
print(f"\nAUC CatBoost (test): {roc_auc_score(y, test_cat_preds):.4f}")
print(f"AUC XGBoost (test): {roc_auc_score(y, test_xgb_preds):.4f}")
print(f"AUC STACKING (test): {roc_auc_score(y, final_predictions):.4f}")


AUC CatBoost (test): 0.8788
AUC XGBoost (test): 0.8692
AUC STACKING (test): 0.8773


In [30]:
print("\n" + "="*60)
print("EVALUACIÓN CON THRESHOLD OPTIMIZADO")
print("="*60)

# Costos de negocio
cost_FN = 10000  # Costo de no detectar un default (pérdida del préstamo)
cost_FP = 1000   # Costo de rechazar un buen cliente (oportunidad perdida)

# Buscar threshold óptimo
thresholds = np.linspace(0, 1, 101)
losses = []

for t in thresholds:
    y_pred = (final_predictions >= t).astype(int)

    FP = np.sum((y == 0) & (y_pred == 1))
    FN = np.sum((y == 1) & (y_pred == 0))

    loss = FP * cost_FP + FN * cost_FN
    losses.append(loss)

optimal_threshold = thresholds[np.argmin(losses)]
min_loss = min(losses)

print(f"\n💰 Análisis de Costos:")
print(f"Threshold óptimo: {optimal_threshold:.4f}")
print(f"Pérdida mínima: ${min_loss:,.0f}")

# Predicciones con threshold optimizado
y_test_pred = (final_predictions >= optimal_threshold).astype(int)

# Métricas de clasificación
print(f"\n📈 Métricas de Clasificación (threshold={optimal_threshold:.4f}):")

# Confusion matrix manual
TP = np.sum((y == 1) & (y_test_pred == 1))
FP = np.sum((y == 0) & (y_test_pred == 1))
TN = np.sum((y == 0) & (y_test_pred == 0))
FN = np.sum((y == 1) & (y_test_pred == 0))

FPR = FP / (FP + TN) if (FP + TN) > 0 else 0
TPR = TP / (TP + FN) if (TP + FN) > 0 else 0
Precision = TP / (TP + FP) if (TP + FP) > 0 else 0
Recall = TPR

print(f"FPR (False Positive Rate): {FPR:.4f}")
print(f"TPR (Recall): {TPR:.4f}")
print(f"Precision (PPV): {Precision:.4f}")
print(f"AUC: {roc_auc_score(y, final_predictions):.5f}")

# Classification report
print(f"\n📋 Classification Report:")
print(classification_report(y, y_test_pred, target_names=['No Default', 'Default']))

# Confusion matrix
cm = confusion_matrix(y, y_test_pred)
print(f"\n📊 Matriz de Confusión:")
print(f"                 Predicho")
print(f"               No Def  Default")
print(f"Real No Def    {cm[0, 0]:6d}   {cm[0, 1]:6d}")
print(f"Real Default   {cm[1, 0]:6d}   {cm[1, 1]:6d}")

# Análisis de costos detallado
total_cost_optimal = FP * cost_FP + FN * cost_FN
total_cost_default = np.sum(y == 1) * cost_FN  # Si rechazamos a todos
savings = total_cost_default - total_cost_optimal

print(f"\n💵 Análisis Financiero:")
print(f"Costo total (threshold óptimo): ${total_cost_optimal:,.0f}")
print(f"Costo si rechazamos a todos: ${total_cost_default:,.0f}")
print(f"Ahorro con el modelo: ${savings:,.0f} ({(savings/total_cost_default)*100:.1f}%)")
print(f"\nFalsos Positivos: {FP} (costo: ${FP * cost_FP:,.0f})")
print(f"Falsos Negativos: {FN} (costo: ${FN * cost_FN:,.0f})")


EVALUACIÓN CON THRESHOLD OPTIMIZADO

💰 Análisis de Costos:
Threshold óptimo: 0.3600
Pérdida mínima: $90,096,000

📈 Métricas de Clasificación (threshold=0.3600):
FPR (False Positive Rate): 0.2193
TPR (Recall): 0.8187
Precision (PPV): 0.2614
AUC: 0.87728

📋 Classification Report:
              precision    recall  f1-score   support

  No Default       0.98      0.78      0.87    230302
     Default       0.26      0.82      0.40     21835

    accuracy                           0.78    252137
   macro avg       0.62      0.80      0.63    252137
weighted avg       0.92      0.78      0.83    252137


📊 Matriz de Confusión:
                 Predicho
               No Def  Default
Real No Def    179796    50506
Real Default     3959    17876

💵 Análisis Financiero:
Costo total (threshold óptimo): $90,096,000
Costo si rechazamos a todos: $218,350,000
Ahorro con el modelo: $128,254,000 (58.7%)

Falsos Positivos: 50506 (costo: $50,506,000)
Falsos Negativos: 3959 (costo: $39,590,000)
